In [1]:
#!pip install conorm
from bioinfokit.analys import norm, get_data
import bioinfokit
import conorm # for tmm normalisation

In [2]:
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
from pathlib import Path
import functools
# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
#loading the count data

path1=Path("/home/znazari/machine_learning_ppmi")

quant_rna1=pd.read_csv(path1/"ordered_bl_matrix_ir2_count.txt", header=None,index_col=[0] )
quant_rna=quant_rna1.rename(columns=quant_rna1.iloc[0]).drop(quant_rna1.index[0]).reset_index(drop=True)

In [4]:
#loading the counts data

path_=Path("/scratch/znazari/PPMI_ver_sep2022/study_data/Subject_Characteristics")

data_ps=pd.read_csv(path_/"Participant_Status.csv", header=None )
ps_data=data_ps.rename(columns=data_ps.iloc[0]).drop(data_ps.index[0]).reset_index(drop=True)
ps_data

,PATNO,COHORT,COHORT_DEFINITION,ENROLL_DATE,ENROLL_STATUS,STATUS_DATE,ENROLL_AGE,INEXPAGE,AV133STDY,PPMI_ONLINE_ENROLL,...,COMMENTS,CONDATE,ENRLPINK1,ENRLPRKN,ENRLSRDC,ENRLHPSM,ENRLRBD,ENRLLRRK2,ENRLSNCA,ENRLGBA
0,3000,2,Healthy Control,02/2011,enrolled,05/2021,69.1,INEXHC,0,NO,...,NaN,06/2021,0,0,0,0,0,0,0,0
1,3001,1,Parkinson's Disease,03/2011,enrolled,09/2021,65.1,INEXPD,0,NO,...,NaN,06/2021,0,0,1,0,0,0,0,0
2,3002,1,Parkinson's Disease,03/2011,enrolled,09/2021,67.6,INEXPD,0,NO,...,NaN,06/2021,0,0,1,0,0,0,0,0
3,3003,1,Parkinson's Disease,04/2011,enrolled,01/2022,56.7,INEXPD,0,NO,...,NaN,06/2021,0,0,1,0,0,0,0,0
4,3004,2,Healthy Control,04/2011,enrolled,01/2022,59.4,INEXHC,0,YES,...,NaN,06/2021,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,162994,1,Parkinson's Disease,NaN,screened,07/2022,NaN,INEXPD,0,NO,...,NaN,NaN,0,0,1,0,0,0,0,0
2347,163265,1,Parkinson's Disease,NaN,screened,07/2022,NaN,INEXPD,0,YES,...,NaN,NaN,0,0,1,0,0,0,0,0
2348,164900,1,Parkinson's Disease,NaN,screened,07/2022,NaN,INEXPD,0,NO,...,NaN,NaN,0,0,1,0,0,0,0,0
2349,167222,1,Parkinson's Disease,NaN,screened,08/2022,NaN,INEXPD,0,NO,...,NaN,NaN,0,0,1,0,0,0,0,0


In [5]:
ps_data["PATNO"]

0         3000
1         3001
2         3002
3         3003
4         3004
         ...  
2346    162994
2347    163265
2348    164900
2349    167222
2350    168306
Name: PATNO, Length: 2351, dtype: object

In [6]:
quant_rna.head()

,Geneid,10874,12499,12593,13039,13424,14281,14331,14426,15761,...,59614,60002,60015,60024,60036,60095,60171,65002,65003,85242
0,ENSG00000223972.4,10,10,12,7,10,5,7,4,4,...,3,9,9,8,11,2,1,5,6,6
1,ENSG00000227232.4,192,418,276,252,292,235,162,209,128,...,348,158,205,210,512,211,134,168,235,432
2,ENSG00000243485.2,0,0,2,0,0,0,0,0,0,...,0,2,0,0,1,1,0,0,0,0
3,ENSG00000237613.2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ENSG00000268020.2,1,0,0,0,0,1,0,0,1,...,2,0,0,0,0,0,0,0,0,0


In [7]:
original_columns=list(quant_rna.columns)
original_columns[0]

'Geneid'

In [8]:
#since the first elemnt of the list was an string, we remove it this way. Howver, we must remeber to run this just 
#just once, otherwise we reach to extra elimination.
original_columns.remove(original_columns[0])


In [9]:
len(original_columns)

1128

In [10]:
ps_data[["PATNO","ENRLGBA"]]

,PATNO,ENRLGBA
0,3000,0
1,3001,0
2,3002,0
3,3003,0
4,3004,0
...,...,...
2346,162994,0
2347,163265,0
2348,164900,0
2349,167222,0


In [11]:
big_patno=list(ps_data["PATNO"])

In [12]:
contains=[]
not_contains=[]
for i in original_columns:
    if i in big_patients:
        contains+=[i]
    else:
        not_contains+=[i]

NameError: name 'big_patients' is not defined

In [ ]:
#great idea to set the geneid as an index counter so to avoid its confliction with the rest.
dff1 = quant_rna.set_index('Geneid')

In below we do the following steps:

1- we filter the rows in which their average is less than 5.

2- we apply tmm normalisation

3- we take log2 of the data

In [ ]:
#finding average of each row, and making a new column for the average
i_average=dff1.mean(axis=1)
i_average
dff1["average"]=i_average

In [ ]:
#we filter all the rows that their average is less than 5,then we drop the nan value rows
filter = dff1["average"]>5
# filtering data
dff1.where(filter, inplace = True)
dff_drop=dff1.dropna()
dff_drop

In [ ]:
# to add one so that later when we take log2 we do not end with conflict
dff=dff_drop.add(1)

In [ ]:
#Trimmed Means of M-values (tmm)
dff_tmm = conorm.tmm(dff)
dff_tmm

In [ ]:
#log2 normalisation
log_tmm=np.log2(dff_tmm)
data_log_tmm=pd.DataFrame(log_tmm)
data_log_tmm

In [ ]:
#the first step to get the cpm norm is these two lines
#nm = norm()
#nm.cpm(df=dff)
#here is the cpm norm taking step 
#cpm_df = nm.cpm_norm
#cpm_df.head(5)

In [ ]:
#loading the quant diagnosis: prediction

quant_predict1=pd.read_csv(path1/"the_diagnosis_count_ir2.txt", header=None ,index_col=[0])
quant_predict=quant_predict1.rename(columns=quant_predict1.iloc[0]).drop(quant_predict1.index[0]).reset_index(drop=True)

In [ ]:
#transposed matrix
t_quant=data_log_tmm.T
t_quant

In [ ]:
searchfor = ['Healthy Control', "Parkinson's Disease"]


In [ ]:
#we just pick those that are either health control or parkinson'd diseases
quant_p=quant_predict[quant_predict['COHORT_DEFINITION'].astype(str).str.contains('|'.join(searchfor))]

In [ ]:
quant_new_predict=quant_p['BL_PATNO']

In [ ]:
r_quant_patients=[int(x) for x in quant_new_predict]

In [ ]:
#here we select those rna-seq data that includes both health control and parkinson 
selected_quant_rna=data_log_tmm[r_quant_patients]

In [ ]:
select_quant_rna=selected_quant_rna.T

In [ ]:
quant_p['COHORT_DEFINITION'].replace(['Healthy Control', "Parkinson's Disease"],
                        [0, 1], inplace=True)

In [ ]:
#here we make the model
bst_q = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='reg:logistic')


In [ ]:
#here we repeat the machine learning procedure for k times
k=1
g0=0
for i in range(k):
    X_trainq, X_testq, y_trainq, y_testq = train_test_split(select_quant_rna,quant_p['COHORT_DEFINITION'], test_size=.3)
    bst_q.fit(X_trainq, y_trainq)
    preds_q = bst_q.predict(X_testq)
    accuracy = accuracy_score(y_testq, preds_q)
    g= (accuracy * 100.0)
    g+=g0
    g0=g
print(round(g/k,2))

In [ ]:
select_quant_rna

In [ ]:
cplr=bst_q.predict_proba(X_testq)[:,1]

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_curve

import matplotlib.pyplot as plt

In [ ]:
print('Accuracy:  ', accuracy_score(y_testq,bst_q.predict(X_testq)))
print('Precision: ', precision_score(y_testq,bst_q.predict(X_testq)))
print('Recall:    ', recall_score(y_testq,bst_q.predict(X_testq)))
print('F1:        ', f1_score(y_testq,bst_q.predict(X_testq)))
print('ROC-AUC:   ', roc_auc_score(y_testq,bst_q.predict(X_testq)))
print('\nConfusion Matrix\n{0}'.format(confusion_matrix(y_testq,bst_q.predict(X_testq))))

In [ ]:
print('Accuracy:  ', accuracy_score(y_trainq,bst_q.predict(X_trainq)))
print('Precision: ', precision_score(y_testq,bst_q.predict(X_testq)))
print('Recall:    ', recall_score(y_testq,bst_q.predict(X_testq)))
print('F1:        ', f1_score(y_testq,bst_q.predict(X_testq)))
print('ROC-AUC:   ', roc_auc_score(y_trainq,bst_q.predict(X_trainq)))
print('\nConfusion Matrix\n{0}'.format(confusion_matrix(y_testq,bst_q.predict(X_testq))))

*confusion matrix* TN, FP; FN, TP

In [ ]:
roc = roc_curve(y_trainq,bst_q.predict_proba(X_trainq)[:,1])
pr  = precision_recall_curve(y_trainq,bst_q.predict_proba(X_trainq)[:,1])

f = plt.figure(figsize=(20,7))
ax = f.add_subplot(121)
ax.plot(roc[0],roc[1])
ax.set_ylabel('True Positive Rate')
ax.set_xlabel('False Positive Rate')
ax.set_title('ROC Curve')
ax.grid(which='both')
ax = f.add_subplot(122)
ax.plot(pr[1],pr[0])
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')
ax.set_title('PR-curve')
ax.grid(which='both')
plt.show()

In [ ]:
#pip install pydeseq2

In [ ]:
import os
import pickle as pkl

from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import load_example_data